# @Xiufan: Please update the aws-braket-sdk and the qiskit-braket-provider to their latest package version

### To update the packages, just run this command in your terminal please

pip install amazon-braket-sdk==1.72.2 pip install qiskit-braket-provider==0.1.1

#### Running a bell circuit on different backends

In [ ]:
import numpy as np
import time

from braket.circuits import Circuit
from qiskit_braket_provider import AWSBraketProvider
from braket.aws import AwsSession

from qiskit import QuantumCircuit
from qiskit.primitives import BackendEstimator, Estimator
from qiskit.quantum_info.operators import SparsePauliOp
from qiskit.primitives import BackendEstimator

import os
os.environ["KMP_DUPLICATE_LIB_OK"] = "True"

# TODO: Please adjust the S3 bucket to one of yours

In [ ]:
aws_session = AwsSession(default_bucket="amazon-braket-us-west-1-lukasvoss") # TODO: Please replace with your S3 bucket

In [ ]:
batchsize = 50
n_shots = 10_000

#### Run with native Braket SV Backend

In [ ]:
backend = AWSBraketProvider().get_backend('SV1')

device = backend._device
device

In [ ]:
braket_bell_circ = Circuit().h(0).cnot(0, 1)
print(braket_bell_circ)

In [ ]:
start_time = time.time()
job_braket_native = device.run_batch(
    [braket_bell_circ] * batchsize,
    shots=n_shots,
)
run_batch_time = time.time() - start_time
print('--- {} seconds ---'.format(round(run_batch_time, 2)))
print('Execution time per circuit:', round(run_batch_time / batchsize, 2), 'seconds')

#### Run with Braket Provider + **Estimator Primitive**

In [ ]:
qiskit_bell_circ = QuantumCircuit(2)
qiskit_bell_circ.h(0)
qiskit_bell_circ.cx(0, 1)

qiskit_bell_circ.draw("mpl")

Formerly took 532 secs to complete

In [ ]:
backend = AWSBraketProvider().get_backend('SV1')
estimator = BackendEstimator(backend)
qiskit_observables = SparsePauliOp.from_list(([('XX', 0.25), ("IZ", 0.25), ("ZZ", 0.25), ("XY", 0.25)]))

qiskit_start_time = time.time()
job = estimator.run(
    circuits=[qiskit_bell_circ] * batchsize,
    observables=[qiskit_observables] * batchsize,
    shots=n_shots
)
job.result().values # This is the time-consuming line

qiskit_run_batch_time = time.time() - qiskit_start_time
print('--- {} seconds ---'.format(round(qiskit_run_batch_time, 2)))
print('Execution time per circuit:', round(qiskit_run_batch_time / (batchsize * 3), 2), 'seconds')

In [ ]:
qiskit_expvals = job.result().values

#### Use ``Estimator`` from Qiskit

In [ ]:
start_time = time.time()
job = Estimator().run(
    circuits=[qiskit_bell_circ] * batchsize,
    observables=[qiskit_observables] * batchsize,
    shots=n_shots
)
qiskit_expvals_2 = job.result().values
run_batch_time = time.time() - start_time
print('--- {} seconds ---'.format(round(run_batch_time, 2)))
# Three non-commuting parts of the Hamiltonian lead to three measured circuits per Hamiltonian
print('Execution time per circuit:', round(run_batch_time / (batchsize * 3), 2), 'seconds')

In [ ]:
qiskit_expvals_2 - qiskit_expvals

In [ ]:
max(qiskit_expvals_2 - qiskit_expvals)

#### Running batch of qiskit circuits with SV1 backend provider

In [ ]:
backend = AWSBraketProvider().get_backend('SV1')

In [ ]:
qiskit_start_time = time.time()
job = backend.run(
    [qiskit_bell_circ] * batchsize,
    shots=n_shots
)
result = job.result() # This is the time-consuming line

qiskit_run_batch_time = time.time() - qiskit_start_time
print('--- {} seconds ---'.format(round(qiskit_run_batch_time, 2)))
print('Execution time per batch:', round(qiskit_run_batch_time / batchsize, 2), 'seconds')